In [ ]:
import re
import numpy as np

In [ ]:
def evaluate(op1, op2, operator):        # this function evaluates the arithmetic operations 

# op1 and op2: the first and second operands of the arithmetic operation respectively
# operator: the operator of the arithmetic operation

    operand1 = float(op1)
    operand2 = float(op2)
    if operator == '^':
        return operand1 ** operand2
    if operator == '*':
        return operand1 * operand2
    if operator == '/':
        if operand2 == 0:
            return np.nan
        return operand1 / operand2
    if operator == '+':
        return operand1 + operand2
    if operator == '-':
        return operand1 - operand2

In [ ]:
def funParsing(FunctionString):
    # this function convert the function string into array of elements (operators, operands and x char)
    # FunctionString: the input function expression as a string
    # FunElements: array of separated elements (operators, operands and x char)
    FunStr=FunctionString.replace(' ','')
    FunElements=list(FunStr)
    start=FunElements[0]
    if start=='-': 
        if FunElements[1] ==('*') or FunElements[1] ==('/') or FunElements[1] ==('^'): 
            FunElements[0]='-1'
        else:
            FunElements[0 : 2] = [''.join(FunElements[0 : 2])]
    return FunElements

In [ ]:
def funStrIsValid(funcString):
    # this function validate the function string as an valid function expression
    # funcString: the input string for function expression
    # True: the output in case of valid expression  
    # False: the output in case of invalid expression
    FunStr=funcString.replace(' ','')
    FunElements=list(FunStr)
    start=FunElements[0]
    end=FunElements[len(FunElements)-1]
    if start =='*' or start=='/'or start =='^':
        return False

    if end==('*') or end ==('/') or end ==('^') or end ==('+') or end ==('-'):
        return False
    return True

In [ ]:
def evalByIdx(fArray, index):
    # this function reduce the function Array by evaluating the operation of the given index in the function array
    # FunArray: array of expression elements
    # index   : index of the operator to evaluate its arithmetic operation
    # NewFArray: array of the resulted expression elements
    
    oldFArray = fArray.copy()
    NewFArray = []
    result = str(evaluate(oldFArray[index - 1], oldFArray[index + 1], oldFArray[index]))
    for j, value in enumerate(oldFArray):
        if j == index or j == index + 1:
            continue
        if j == index - 1:
            NewFArray.append(result)
        else:
            NewFArray.append(value)

    return NewFArray

In [ ]:
def evaluateExpersion(FunArray, xValue):
     # this function calculates function expression for a given x value
     # FunArray: array of separated elements (operators, operands and x char)
     # xValue: the value of x to calculate the function at that point
     # functionValue: the value of function at xValue point

    newFunArray = FunArray.copy()

    for i in range(len(newFunArray)):
        if newFunArray[i]=='x':
            newFunArray[i] = xValue
        if newFunArray[i]=='-x':
            newFunArray[i] = -xValue
            
            

    while '^' in newFunArray:                         # evaluate ^ from right to left
        # looping in reverse
        for idx,element in enumerate(newFunArray[::-1]):
            if element == '^':
                newFunArray = evalByIdx(newFunArray, len(newFunArray)-idx-1)
                break

    while '*' in newFunArray or '/' in newFunArray:      # evaluate *, / from left to right
        for idx,element in enumerate(newFunArray):
            if element == '*' or element == '/':
                newFunArray = evalByIdx(newFunArray, idx)
                break

    while '+' in newFunArray or '-' in newFunArray:        # evaluate +, - from left to right
        for idx,element in enumerate(newFunArray):
            if element == '+' or element == '-':
                newFunArray = evalByIdx(newFunArray, idx)
                break

    functionValue = newFunArray[0]
    return functionValue

GUI

In [ ]:
import sys
import random
from decimal import Decimal
from PySide2 import QtWidgets, QtCore, QtGui
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT
from matplotlib.figure import Figure

In [ ]:
class MplCanvas(FigureCanvasQTAgg):
    # This class is for plot widget and figure of the evaluated data
    def __init__(self, parent=None, width=6, height=5, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        self.axes.grid(True)
        super(MplCanvas, self).__init__(fig)

In [ ]:
class PlotterApp(QtWidgets.QMainWindow):
    # The main appliction Class where the whole GUI is built
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Function Plotter")     # set initial window shape
        self.setGeometry(300, 400, 600, 600)
        self.canvas = MplCanvas(self, 6, 5, 100)    # create initial figure and toolbar for the plotter
        self.toolbar = NavigationToolbar2QT(self.canvas, self)
        self.createDataGroupBox()               # container for the input data        
        grid = QtWidgets.QGridLayout()          # organize the layout of the main parts of the GUI
        grid.addWidget(self.toolbar, 0, 0)
        grid.addWidget(self.canvas, 1, 0)
        grid.addWidget(self.dataGroupBox, 2, 0)
        widget = QtWidgets.QWidget()
        widget.setLayout(grid)
        self.setCentralWidget(widget)
        self.center()
        self.pushButton_plot.clicked.connect(self.plotFunction) # connect signals and slots
        
    def drawGraph(self, xVaues, yValues):
        # this function takes the x,y values arrays to plot it on the figure directly       
        #  xVaues: Array of input data on the X-axis
        #  yValues: Array of the corresponding data on the Y-axis
        self.canvas.axes.cla()
        self.canvas.axes.plot(xVaues, yValues, 'r')
        self.canvas.axes.grid(True)
        self.canvas.draw()

    def plotFunction(self):
        # This function is called on pushbutton click event, it takes the needed arguments from different elemnts in
        # the GUI, making sure of their validity then calculate the function to plot the data, The main function is
        # organised here

        functionString = None   # initialize main input data
        xMin = None
        xMax = None
       
        if self.lineEdit_function.text():        # verify None empty input data
            functionString = self.lineEdit_function.text()
        else:
            QtWidgets.QMessageBox.about(self, "empty Function", "you have to enter a function expression")
            return
        if self.lineEdit_xMin.text() and self.lineEdit_xMax.text():
            xMin = float(Decimal(self.lineEdit_xMin.text()))
            xMax = float(Decimal(self.lineEdit_xMax.text()))
            if xMin >= xMax:
                QtWidgets.QMessageBox.about(self, "Wrong limits", "x min limit should be less than x max limit")
                return
        else:
            QtWidgets.QMessageBox.about(self, "empty X limits", "you have to enter the two X limit values")
            return

        if functionString != None and xMin != None and xMax != None:   # do the main stuff after checking none empty input data

            
            if(not funStrIsValid(functionString)):      # check for Valid input Function Expression
                QtWidgets.QMessageBox.about(self, "Wrong Expression", "Not a valid Expression")
            else:
                # partitioning the complex expression into simple elements  (operators and operands) stored in array
                expressionArray = funParsing(functionString)
                xArray = np.linspace(xMin, xMax, 1000)
                yArray = []

                
                for x in xArray:                      # calculate the function for each x value from the X-axis
                    y = evaluateExpersion(expressionArray, x)
                    yArray.append(float(y))
                self.drawGraph(xArray, yArray)

    def createDataGroupBox(self):
        # build the container for input data
        self.dataGroupBox = QtWidgets.QGroupBox("curve input data", self)
        grid = QtWidgets.QGridLayout()

        self.label_function = QtWidgets.QLabel("function",self)
        grid.addWidget(self.label_function, 0, 0)

        self.lineEdit_function = QtWidgets.QLineEdit(self)
        self.lineEdit_function.setPlaceholderText("function")
        grid.addWidget(self.lineEdit_function, 0, 1, 1, -1)

        self.label_xMin = QtWidgets.QLabel("X min",self)
        grid.addWidget(self.label_xMin, 1, 0)

        self.lineEdit_xMin = QtWidgets.QLineEdit(self)
        self.lineEdit_xMin.setPlaceholderText("X min")
        self.lineEdit_xMin.setValidator(QtGui.QDoubleValidator())
        grid.addWidget(self.lineEdit_xMin, 1, 1)

        self.label_xMax = QtWidgets.QLabel("X max",self)
        grid.addWidget(self.label_xMax, 1, 2)

        self.lineEdit_xMax = QtWidgets.QLineEdit(self)
        self.lineEdit_xMax.setPlaceholderText("X max")
        self.lineEdit_xMax.setValidator(QtGui.QDoubleValidator())
        grid.addWidget(self.lineEdit_xMax, 1, 3)

        self.pushButton_plot = QtWidgets.QPushButton("plot",self)
        grid.addWidget(self.pushButton_plot, 1, 4)

        self.dataGroupBox.setLayout(grid)



    def center(self):
        frame = self.frameGeometry()
        centerPoint = QtWidgets.QDesktopWidget().availableGeometry().center()
        frame.moveCenter(centerPoint)
        self.move(frame.topLeft())


 # start the main application
if __name__ == "__main__":
   
    myApp = QtWidgets.QApplication(sys.argv)
    myplotter = PlotterApp()
    myplotter.show()
    myApp.exec_()

    